In [ ]:
!pip install pyspark

In [11]:
import os
from time import time
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import split, col, count, size, format_string, input_file_name, element_at
from pyspark.sql.types import StructType

dataset_path = "BDAchallenge2324"
output_path = "results"

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [12]:
def read_csv(year, station):
    dataframe = spark.read.format('csv') \
        .option('header', 'true') \
        .load('{}/{}/{}'.format(dataset_path, year, station)) \
        .withColumn('year', element_at(split(input_file_name(), '/'), -2).cast("string")) \
        .withColumn('station', element_at(split(input_file_name(), "/"), -1).cast("string")) \
        .withColumn('station', split(col('station'), '.csv')[0])
    return dataframe

In [13]:
def export_csv(dataframe, file_name):
    file_path = "{}/{}".format(output_path, file_name)
    if not os.path.exists(file_path):
        dataframe.coalesce(1).write.format("csv").option("header", "true").save(file_path)
    else:
        dataframe.coalesce(1).write.format("csv").mode("overwrite").option("header", "true").save(file_path)
    print("File .csv esportato con successo in: {}/{}".format(output_path, file_name))

In [14]:
def export_txt(file_name, result):
    with open('{}/{}'.format(output_path, file_name), "w") as file:
        if isinstance(result, list):
           file.write("\n".join(result))
        else:
           file.write(result)
    print("File .txt esportato con successo in: {}/{}".format(output_path, file_name))

# Assignment 1

#### Stampare il numero di misurazioni effettuate per ogni anno per ogni stazione (ordinato per anno e stazione

In [15]:
def first_assignment(dataframe):
    rows = []
    output_dataframe = dataframe.select("year", "station") \
        .groupBy("year", "station") \
        .agg(count("*").alias("measures_count")) \
        .orderBy("year", "station") 
    for row in output_dataframe.collect():
        row_values = "{}, {}, {}".format(row["year"], row["station"], row["measures_count"])
        print(row_values)
        rows.append(row_values)
    export_txt("task1.txt", rows)
    export_csv(output_dataframe, "task1.csv")

# Assignment 2

#### Stampare le prime 10 temperature (TMP) con il maggior numero di occorrenze ed il relativo conteggio registrate nell’area         evidenziata (ordinate per numero di occorrenze e temperatura)

In [16]:
def second_assignment(dataframe):
    rows = []
    output_dataframe = dataframe \
        .filter((col("LATITUDE").between(30, 60)) & (col("LONGITUDE").between(-135, -90))) \
        .groupBy("TMP") \
        .agg(count("*").alias("TMP_count")) \
        .orderBy(col("TMP_count").desc(), col("TMP").desc()) \
        .limit(10)
    for row in output_dataframe.collect():
        row_values = "[(60,-135);(30,-90)], {}, {}".format(float(row["TMP"][1:].replace(',', '.')), row["TMP_count"])
        print(row_values)
        rows.append(row_values)
    export_txt("task2.txt", rows)
    export_csv(output_dataframe, "task2.csv")

# Assignment 3

#### Stampare la stazione con la velocità in nodi che occorre più volte ed il relativo conteggio (ordinando per conteggio, velocità e stazione)

In [17]:
def third_assignment(dataframe):
    output_dataframe = dataframe \
        .withColumn('WDN_speed', split(col('WND'), ',')[1]) \
        .groupBy('station', 'WDN_speed') \
        .agg(count('*').alias('WND_speed_count')) \
        .orderBy(col("WND_speed_count").desc(), col("WDN_speed").desc(), col("station").asc()) \
        .limit(1) 
    result = "{}, {}, {}".format(output_dataframe.collect()[0]["station"], \
                                 output_dataframe.collect()[0]["WDN_speed"], \
                                 output_dataframe.collect()[0]["WND_speed_count"])
    print(result)
    export_txt("task3.txt", result)
    export_csv(output_dataframe, "task3.csv")

# Main

In [18]:
if __name__ == "__main__":
    starting_time = time()
    schema = StructType([])
    total_dataframe = spark.createDataFrame([], schema)
    for root, dirs, files in sorted(os.walk((dataset_path))):
        for file in sorted(files):
            if (file == ".DS_Store"):
                continue
            dataframe = read_csv(os.path.basename(root), file)
            total_dataframe = total_dataframe.unionByName(dataframe, allowMissingColumns=True)
    first_assignment(total_dataframe)
    second_assignment(total_dataframe)
    third_assignment(total_dataframe)
    print("Tempo di completamento: {} seconds.".format(time() - starting_time))

2000, 99999994988, 316
2000, 99999994989, 316
2000, 99999994991, 316
2000, 99999994992, 316
2000, 99999994993, 316
2000, 99999994994, 316
2001, 99999994988, 375
2001, 99999994989, 375
2001, 99999994992, 375
2001, 99999994993, 375
2001, 99999994994, 375
2002, 01092099999, 2805
2002, 02328099999, 1
2002, 02508099999, 4
2002, 02602099999, 2698
2002, 02869099999, 16848
2002, 03118099999, 2
2002, 03604099999, 8413
2002, 04212099999, 6
2002, 06021099999, 1381
2003, 02602099999, 2706
2003, 15318099999, 1
2003, 57328099999, 2872
2003, 68377099999, 1251
2003, 72410499999, 18293
2003, 89266099999, 652
2003, 95658099999, 2822
2003, 99640099999, 8466
2003, 99689099999, 3
2004, 02328099999, 1
2004, 02602099999, 2603
2004, 57328099999, 2899
2004, 68377099999, 1322
2004, 71393099999, 392
2004, 72410499999, 18616
2004, 95658099999, 2839
2004, 99640099999, 8594
2004, 99689099999, 2
File .txt esportato con successo in: results/task1.txt
File .csv esportato con successo in: results/task1.csv
[(60,-135);(